In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_chfusd = pd.read_csv('/content/CHF-USD Exchange Rate Clean.csv')
df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [ ]:
df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_chfusd['Date'] = pd.to_datetime(df_chfusd.iloc[:, 0], infer_datetime_format=True)

In [ ]:
df_trends.iloc[:,1:] = df_trends.iloc[:,1:].diff()
df_trends

,Date,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,0.0,16.0,9.0,2.0,-1.0,10.0,-27.0,17.0,31.0,1.0,-2.0,0.0,1.0,-4.0,-1.0,-2.0,-6.0,2.0,2.0,0.0,-1.0,-6.0,7.0,20.0,0.0,0.0,1.0,19.0,0.0,8.0,0.0,-58.0,-1.0,3.0,-24.0,0.0,-3.0,10.0,0.0,...,-2.0,-13.0,-34.0,-13.0,18.0,19.0,6.0,0.0,-5.0,14.0,-5.0,25.0,11.0,2.0,5.0,34.0,15.0,-11.0,0.0,24.0,-13.0,0.0,-8.0,6.0,6.0,0.0,0.0,5.0,-3.0,35.0,10.0,0.0,-4.0,8.0,0.0,8.0,0.0,-16.0,6.0,10.0
2,2004-01-03,0.0,0.0,-10.0,-3.0,0.0,-6.0,0.0,-19.0,-31.0,-1.0,-3.0,0.0,-1.0,-1.0,0.0,-3.0,4.0,-2.0,0.0,0.0,0.0,5.0,-4.0,-16.0,0.0,10.0,-1.0,-6.0,0.0,-6.0,0.0,2.0,10.0,-7.0,30.0,0.0,0.0,-6.0,0.0,...,6.0,8.0,4.0,-1.0,-21.0,2.0,1.0,0.0,-4.0,-16.0,25.0,12.0,-21.0,-1.0,-6.0,-17.0,-3.0,0.0,0.0,-16.0,2.0,0.0,-8.0,-2.0,0.0,0.0,17.0,-5.0,8.0,-25.0,-2.0,0.0,8.0,7.0,0.0,-14.0,0.0,13.0,7.0,32.0
3,2004-01-04,19.0,-12.0,2.0,7.0,1.0,3.0,22.0,12.0,0.0,1.0,0.0,26.0,1.0,2.0,0.0,11.0,-5.0,1.0,1.0,2.0,0.0,0.0,-3.0,-3.0,0.0,-19.0,1.0,-8.0,0.0,2.0,0.0,7.0,-5.0,5.0,-38.0,0.0,0.0,-1.0,1.0,...,-5.0,-6.0,13.0,2.0,8.0,-16.0,3.0,14.0,5.0,7.0,3.0,-36.0,-25.0,2.0,1.0,23.0,8.0,5.0,10.0,-6.0,8.0,27.0,0.0,-5.0,7.0,0.0,-17.0,4.0,-2.0,1.0,5.0,0.0,-14.0,-9.0,0.0,7.0,0.0,18.0,5.0,-39.0
4,2004-01-05,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,-2.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465,2021-09-13,-1.0,0.0,4.0,0.0,6.0,32.0,17.0,8.0,3.0,1.0,-2.0,1.0,1.0,-1.0,0.0,0.0,1.0,13.0,2.0,0.0,1.0,2.0,7.0,35.0,0.0,4.0,-1.0,7.0,2.0,-1.0,-1.0,3.0,2.0,10.0,0.0,0.0,3.0,5.0,0.0,...,5.0,0.0,-1.0,-6.0,21.0,12.0,2.0,0.0,-7.0,3.0,5.0,9.0,1.0,2.0,8.0,8.0,2.0,2.0,5.0,15.0,17.0,2.0,-1.0,2.0,2.0,2.0,1.0,2.0,-1.0,10.0,1.0,0.0,6.0,1.0,0.0,2.0,0.0,-8.0,22.0,-2.0
6466,2021-09-14,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,...,-1.0,-1.0,1.0,0.0,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0
6467,2021-09-15,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,3.0,1.0,-1.0,0.0,-12.0,0.0,0.0,-1.0,1.0,-1.0,4.0,11.0,-1.0,0.0,0.0,7.0,4.0,1.0,1.0,3.0,-2.0,0.0,-1.0,2.0,7.0,1.0,2.0,0.0,0.0,-4.0,0.0,0.0,0.0,-7.0,-1.0,0.0,0.0,0.0,2.0,3.0
6468,2021-09-16,0.0,0.0,0.0,-1.0,-2.0,-5.0,-4.0,-1.0,-1.0,-

In [ ]:
ts_df = df_chfusd.merge(df_trends, how = 'inner', on = ['Date'])

In [ ]:
ts_df = ts_df.drop(['Adj Close'], axis = 1)
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date'], axis = 1)
ts_df = ts_df.iloc[1:,:]
ts_df

,Log_Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-02,0.000242,0.0,16.0,9.0,2.0,-1.0,10.0,-27.0,17.0,31.0,1.0,-2.0,0.0,1.0,-4.0,-1.0,-2.0,-6.0,2.0,2.0,0.0,-1.0,-6.0,7.0,20.0,0.0,0.0,1.0,19.0,0.0,8.0,0.0,-58.0,-1.0,3.0,-24.0,0.0,-3.0,10.0,0.0,...,-2.0,-13.0,-34.0,-13.0,18.0,19.0,6.0,0.0,-5.0,14.0,-5.0,25.0,11.0,2.0,5.0,34.0,15.0,-11.0,0.0,24.0,-13.0,0.0,-8.0,6.0,6.0,0.0,0.0,5.0,-3.0,35.0,10.0,0.0,-4.0,8.0,0.0,8.0,0.0,-16.0,6.0,10.0
2004-01-05,0.006472,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,-2.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
2004-01-06,0.000081,0.0,34.0,5.0,2.0,2.0,21.0,0.0,-20.0,30.0,5.0,77.0,0.0,0.0,9.0,0.0,7.0,25.0,2.0,3.0,0.0,3.0,-6.0,8.0,46.0,0.0,27.0,-8.0,44.0,0.0,3.0,0.0,41.0,-7.0,16.0,0.0,0.0,8.0,10.0,3.0,...,14.0,-5.0,4.0,16.0,11.0,-11.0,-14.0,12.0,20.0,-18.0,-69.0,22.0,13.0,-2.0,2.0,36.0,10.0,-9.0,-42.0,50.0,48.0,15.0,11.0,2.0,8.0,22.0,39.0,5.0,2.0,1.0,-28.0,15.0,46.0,-4.0,74.0,75.0,0.0,6.0,32.0,30.0
2004-01-07,-0.006150,0.0,-13.0,-3.0,1.0,2.0,-3.0,0.0,-10.0,2.0,-2.0,-21.0,0.0,0.0,-1.0,1.0,7.0,-9.0,-1.0,-2.0,1.0,0.0,6.0,14.0,-16.0,0.0,1.0,1.0,-23.0,0.0,3.0,0.0,-32.0,3.0,-10.0,8.0,0.0,-4.0,-2.0,-1.0,...,-2.0,2.0,-10.0,-9.0,-7.0,29.0,18.0,-3.0,2.0,-2.0,18.0,-2.0,9.0,0.0,2.0,-25.0,17.0,-1.0,2.0,-24.0,-33.0,14.0,-11.0,-9.0,1.0,-12.0,19.0,-2.0,-24.0,-8.0,6.0,-4.0,-22.0,-10.0,-34.0,-50.0,0.0,2.0,3.0,-25.0
2004-01-08,0.010462,0.0,7.0,5.0,1.0,-2.0,16.0,0.0,6.0,-21.0,-1.0,3.0,8.0,0.0,-5.0,0.0,3.0,4.0,3.0,-3.0,0.0,-1.0,-7.0,-8.0,-8.0,2.0,-5.0,-2.0,0.0,0.0,-2.0,0.0,-5.0,0.0,12.0,15.0,0.0,2.0,3.0,1.0,...,2.0,-4.0,5.0,3.0,5.0,-8.0,-11.0,-9.0,8.0,6.0,-8.0,9.0,18.0,-2.0,-3.0,16.0,3.0,2.0,25.0,5.0,18.0,-11.0,0.0,-5.0,-1.0,-10.0,-20.0,-3.0,-13.0,5.0,-5.0,-11.0,21.0,18.0,-40.0,26.0,0.0,-1.0,-7.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,-0.004983,0.0,1.0,-1.0,2.0,1.0,5.0,-7.0,4.0,0.0,1.0,2.0,2.0,1.0,1.0,0.0,2.0,-3.0,1.0,-4.0,1.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,1.0,1.0,0.0,-1.0,0.0,-5.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,...,2.0,1.0,0.0,-2.0,-4.0,5.0,1.0,-1.0,-2.0,-4.0,-1.0,-4.0,1.0,0.0,-1.0,-2.0,4.0,2.0,2.0,2.0,-5.0,4.0,0.0,-3.0,1.0,-2.0,-1.0,0.0,-3.0,4.0,1.0,0.0,1.0,-4.0,0.0,0.0,0.0,0.0,4.0,1.0
2021-09-09,-0.002596,-1.0,-3.0,0.0,-2.0,-1.0,0.0,8.0,-2.0,0.0,0.0,0.0,-1.0,0.0,-2.0,0.0,-1.0,-3.0,3.0,-3.0,0.0,0.0,1.0,2.0,-4.0,2.0,0.0,-2.0,-4.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,1.0,0.0,...,-3.0,-1.0,3.0,-1.0,-13.0,-4.0,2.0,1.0,1.0,-1.0,0.0,3.0,2.0,0.0,-1.0,0.0,-4.0,-1.0,1.0,-2.0,-7.0,-3.0,0.0,-1.0,0.0,5.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,17.0,1.0,-1.0,0.0,1.0,-4.0,-1.0
2021-09-10,0.005799,0.0,-3.0,-1.0,-1.0,-1.0,-4.0,-4.0,-6.0,-2.0,-1.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,-1.0,-4.0,-2.0,-1.0,0.0,-2.0,-1.0,-6.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,-1.0,-

#Stationary Test

In [ ]:
result = adfuller(ts_df['Log_Difference'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -70.997471
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567


In [ ]:
for column in ts_df:
  result = adfuller(ts_df[column])
  print(column)
  print('ADF Statistic: %f' % result[0])
  print('p-value: %f' % result[1])
  print('Critical Values:')
  for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

Log_Difference
ADF Statistic: -70.997471
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
swiss franc
ADF Statistic: -12.628300
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
Europe
ADF Statistic: -11.803074
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
EUR
ADF Statistic: -8.121912
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
Switzerland
ADF Statistic: -16.543600
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
CHF
ADF Statistic: -9.040752
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
NASDAQ
ADF Statistic: -3.924819
p-value: 0.001860
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
S&P500
ADF Statistic: -6.239133
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
VIX
ADF Statistic: -7.667402
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
volatility
ADF Statistic: -10.985516
p-value: 0.000

In [ ]:
stati = []
trend_names = list(ts_df.columns)
adf = []
p_val = []
critical_value = []

In [ ]:
#Storing Results in Data Frame

for column in ts_df:
  result = adfuller(ts_df[column])
  stati.append(result)
  adf.append(result[0])
  p_val.append(result[1])
  critical_value.append(result[4])

In [ ]:
print(len(stati))
print(len(adf))
print(len(p_val))
print(len(critical_value))
print(len(trend_names))

155
155
155
155
155


In [ ]:
stationary_tests = pd.DataFrame(columns=['Trend Name','ADF Statistic','p-value','Critical Values'])
stationary_tests['ADF Statistic'] = adf
stationary_tests['p-value'] = p_val
stationary_tests['Critical Values'] = critical_value
stationary_tests['Trend Name'] = trend_names

In [ ]:
stationary_tests = stationary_tests.iloc[1:,:]

In [ ]:
stationary_tests

,Trend Name,ADF Statistic,p-value,Critical Values
1,swiss franc,-12.628300,1.524413e-23,"{'1%': -3.4317767256971794, '5%': -2.862170445..."
2,Europe,-11.803074,9.232052e-22,"{'1%': -3.431774860464899, '5%': -2.8621696219..."
3,EUR,-8.121912,1.150014e-12,"{'1%': -3.4317764144860807, '5%': -2.862170308..."
4,Switzerland,-16.543600,1.973743e-29,"{'1%': -3.4317723810618164, '5%': -2.862168526..."
5,CHF,-9.040752,5.141489e-15,"{'1%': -3.4317764144860807, '5%': -2.862170308..."
...,...,...,...,...
150,LIBOR,-6.414297,1.857660e-08,"{'1%': -3.431777037044106, '5%': -2.8621705835..."
151,2008,-8.597243,7.023447e-14,"{'1%': -3.4317767256971794, '5%': -2.862170445..."
152,bubble,-9.494268,3.583841e-16,"{'1%': -3.4317767256971794, '5%': -2.862170445..."
153,energy,-12.323636,6.664441e-23,"{'1%': -3.431777037044106, '5%': -2.8621705835..."


In [ ]:
itiiims = stationary_tests['Critical Values']#[1]

In [ ]:
res = []
for key in itiiims.keys() :
    res.append(itiiims[key])

In [ ]:
res[0]['1%']

-3.4317767256971794

In [ ]:
crit_value = []
for i in range(0, len(res), 1):
  crit_value.append(res[i]['5%'])

In [ ]:
stationary_test_table = pd.DataFrame([stationary_tests['Trend Name'], stationary_tests['ADF Statistic'], crit_value], columns=['Trend Name', 'ADF Statistic', '5% Critical Value'])

AttributeError: ignored

In [ ]:
stationary_test_table = stationary_tests['Trend Name']
stationary_test_table['ADF Statistic'] = stationary_tests['ADF Statistic']
stationary_test_table['5% Critical Value'] = crit_value

In [ ]:
stationary_test_table.head()

1    swiss franc
2         Europe
3            EUR
4    Switzerland
5            CHF
Name: Trend Name, dtype: object

In [ ]:
stationary_tests = stationary_tests.drop(['ADF Statistic', 'Critical Values'], axis=1)

In [ ]:
stationary_tests['p-value'] = stationary_tests['p-value'].round(3)

In [ ]:
stationary_tests.head(50)

,Trend Name,p-value
1,swiss franc,0.000
2,Europe,0.000
3,EUR,0.000
4,Switzerland,0.000
5,CHF,0.000
6,NASDAQ,0.002
7,S&P500,0.000
8,VIX,0.000
9,volatility,0.000
10,Germany,0.000


#T-Test

In [ ]:
t_test = stats.ttest_ind(ts_df['Log_Difference'], ts_df['swiss franc'])

In [ ]:
t_test

Ttest_indResult(statistic=-1.5061782797954562, pvalue=0.1320556195701399)

In [ ]:
for column in ts_df:
  print(column, stats.ttest_ind(ts_df['Log_Difference'], ts_df[column]))

Log_Difference Ttest_indResult(statistic=0.0, pvalue=1.0)
swiss franc Ttest_indResult(statistic=-1.5061782797954562, pvalue=0.1320556195701399)
Europe Ttest_indResult(statistic=-0.6826077062551303, pvalue=0.4948718867439973)
EUR Ttest_indResult(statistic=-12.298562990939296, pvalue=1.7153191098947395e-34)
Switzerland Ttest_indResult(statistic=0.28579490781379885, pvalue=0.7750416191394706)
CHF Ttest_indResult(statistic=-10.97526260523667, pvalue=7.4672211131881845e-28)
NASDAQ Ttest_indResult(statistic=-20.473215397885173, pvalue=3.862137866620118e-91)
S&P500 Ttest_indResult(statistic=-7.237335014632759, pvalue=4.941161169694813e-13)
VIX Ttest_indResult(statistic=-8.854252443881121, pvalue=9.982470159978675e-19)
volatility Ttest_indResult(statistic=-6.0344306451279355, pvalue=1.6566154880615723e-09)
Germany Ttest_indResult(statistic=0.5028356436449701, pvalue=0.6150918235738652)
UK Ttest_indResult(statistic=8.40704607902029, pvalue=4.828525087437407e-17)
sterling pound Ttest_indResult(s

In [ ]:
statistic = []
p_value = []

In [ ]:
for column in ts_df:
  t_test = stats.ttest_ind(ts_df['Log_Difference'], ts_df[column])
  statistic.append(t_test[0])
  p_value.append(t_test[1])

In [ ]:
len(statistic)
len(p_value)

155

In [ ]:
t_tests = pd.DataFrame(columns=['Trend Name','Statistic','p-value'])
t_tests['p-value'] = p_value
t_tests['Statistic'] = statistic
t_tests['Trend Name'] = trend_names

In [ ]:
t_tests.sort_values(by=['p-value'], ascending=True).iloc[120:150,:]

,Trend Name,Statistic,p-value
83,consumption,-2.574668,0.010049
67,politics,-2.548748,0.010827
58,labor,-2.510522,0.012072
30,OPEC,-2.461311,0.013861
61,short selling,-2.218057,0.026575
41,stimulus,-2.168104,0.030176
146,eurodollar,-2.146234,0.031880
149,yield curve,-2.116238,0.034351
21,euro,-1.841989,0.065509
151,2008,1.779360,0.075214


#Cointegration Test

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#x = getx() # dataframe of n series for cointegration analysis
#jres = coint_johansen(x, det_order=0, k_ar_diff=1)

In [ ]:
ts_df.isnull().values.any()

False

In [ ]:
jres = coint_johansen(ts_df, det_order=0, k_ar_diff=13)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/vecm.py:608: HypothesisTestWarning: Critical values are only available for time series with 12 variables at most.
  category=HypothesisTestWarning)


In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,1,13)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [ ]:
cointegration_test(ts_df, alpha=0.05)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/vecm.py:608: HypothesisTestWarning: Critical values are only available for time series with 12 variables at most.
  category=HypothesisTestWarning)


Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Log_Difference ::  94168.08  > nan       =>   False
swiss franc ::  92382.49  > nan       =>   False
Europe ::  90638.67  > nan       =>   False
EUR    ::  88958.27  > nan       =>   False
Switzerland ::  87315.12  > nan       =>   False
CHF    ::  85710.09  > nan       =>   False
NASDAQ ::  84119.3   > nan       =>   False
S&P500 ::  82610.44  > nan       =>   False
VIX    ::  81141.77  > nan       =>   False
volatility ::  79718.69  > nan       =>   False
Germany ::  78310.54  > nan       =>   False
UK     ::  76945.64  > nan       =>   False
sterling pound ::  75596.92  > nan       =>   False
GBP    ::  74261.91  > nan       =>   False
Greece ::  72958.57  > nan       =>   False
Italy  ::  71665.1   > nan       =>   False
Spain  ::  70385.67  > nan       =>   False
France ::  69136.53  > nan       =>   False
DAX    ::  67913.09  > nan       =>   False
unemployment ::  66713.2   > nan       =>   

In [ ]:
coint=coint_johansen(ts_df,-1,13)
#coint.trace_stat
#coint.max_eig_stat
#traces = coint.lr1
#maxeig=coint.lr2
cvts = coint.cvt  ## 0: 90%  1:95% 2: 99%
cvms = coint.cvm   ## 0: 90%  1:95% 2: 99%

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/vecm.py:608: HypothesisTestWarning: Critical values are only available for time series with 12 variables at most.
  category=HypothesisTestWarning)


In [ ]:
out = coint_johansen(ts_df, -1, 5)
out

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/vector_ar/vecm.py:608: HypothesisTestWarning: Critical values are only available for time series with 12 variables at most.
  category=HypothesisTestWarning)


In [ ]:
import statsmodels.tsa.stattools as ts 
result=ts.coint(ts_df['Log_Difference'], ts_df['swiss franc'])
result

(-71.2376434052069, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))

In [ ]:
for column in ts_df:
  print(column, ts.coint(ts_df['Log_Difference'], ts_df[column]))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/stattools.py:1432: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  CollinearityWarning)


Log_Difference (-inf, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
swiss franc (-71.2376434052069, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
Europe (-70.98916185633111, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
EUR (-70.89609451165988, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
Switzerland (-71.01337558355574, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
CHF (-70.52308614572947, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
NASDAQ (-71.0115495315948, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
S&P500 (-71.02285689613916, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
VIX (-71.02646837784243, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
volatility (-70.99833321066625, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
Germany (-71.00436136720728, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
UK (-70.93278845510434, 0.0, array([-3.89881365, -3.33745371, -3.04536873]))
sterling pound (-70.9697230353325,

#Cointegration Test on MiniDF

In [ ]:
small_ts_df = ts_df[['Log_Difference', 'climate change', 'IMF']]

In [ ]:
cointegration_test(small_ts_df, 0.05)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Log_Difference ::  1144.66   > 35.0116   =>   True
climate change ::  726.52    > 18.3985   =>   True
IMF    ::  347.03    > 3.8415    =>   True


In [ ]:
ts_df.iloc[:,1:2]

,swiss franc
Date,
2004-01-02,0.0
2004-01-05,-19.0
2004-01-06,0.0
2004-01-07,0.0
2004-01-08,0.0
...,...
2021-09-08,0.0
2021-09-09,-1.0
2021-09-10,0.0


In [ ]:
ts_df.shape[1]

155

In [ ]:
pd.concat([ts_df['Log_Difference'], ts_df.iloc[:,1:5]], axis=1)

,Log_Difference,swiss franc,Europe,EUR,Switzerland
Date,,,,,
2004-01-02,0.000242,0.0,16.0,9.0,2.0
2004-01-05,0.006472,-19.0,-7.0,-4.0,-8.0
2004-01-06,0.000081,0.0,34.0,5.0,2.0
2004-01-07,-0.006150,0.0,-13.0,-3.0,1.0
2004-01-08,0.010462,0.0,7.0,5.0,1.0
...,...,...,...,...,...
2021-09-08,-0.004983,0.0,1.0,-1.0,2.0
2021-09-09,-0.002596,-1.0,-3.0,0.0,-2.0
2021-09-10,0.005799,0.0,-3.0,-1.0,-1.0


In [ ]:
for i in range(1, ts_df.shape[1], 10):

  subset_ts_df = pd.concat([ts_df['Log_Difference'], ts_df.iloc[:,i:i+10]], axis=1)
  cointegration_test(subset_ts_df, 0.05)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Log_Difference ::  3944.84   > 306.8988  =>   True
swiss franc ::  3382.46   > 259.0267  =>   True
Europe ::  2887.73   > 215.1268  =>   True
EUR    ::  2412.67   > 175.1584  =>   True
Switzerland ::  1988.17   > 139.278   =>   True
CHF    ::  1592.19   > 107.3429  =>   True
NASDAQ ::  1216.04   > 79.3422   =>   True
S&P500 ::  876.95    > 55.2459   =>   True
VIX    ::  571.41    > 35.0116   =>   True
volatility ::  292.76    > 18.3985   =>   True
Germany ::  93.11     > 3.8415    =>   True
Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Log_Difference ::  3540.88   > 306.8988  =>   True
UK     ::  2973.38   > 259.0267  =>   True
sterling pound ::  2527.89   > 215.1268  =>   True
GBP    ::  2143.38   > 175.1584  =>   True
Greece ::  1762.24   > 139.278   =>   True
Italy  ::  1400.57   > 107.3429  =>   True
Spain  ::  1055.31   > 79.3422   =>   True
France ::